<a href="https://colab.research.google.com/github/Balrog16/Test/blob/master/MIMII_Fan_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Background**

Fan audio data from MIMII database is played after combining a few files randomly to form two audio records - normal and abnormal, each lasting for 520 sec. This data is played on JBL Go speaker with SITRANS-MS placed in the middle of the speaker. This arrangement is to vibrate the sensor and obtain the stream via BLE using MATLAB scripts. There are two sets of JSON files each for Normal audio and abnormal audio data.


In [1]:
# all imports
import json
import numpy as np
import tensorflow as tf
from numpy import savetxt
import matplotlib.pyplot as plt

In [2]:
# code to shuffle 
# due credits to Josh Bleecher Snyder
# https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
def shuffle_in_unison_scary(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [3]:
class SMSData:
    def __init__(self, fileName, label):
        self.fileName = fileName
        f = open(self.fileName)
        data = json.load(f)
        self.accelX = np.array(data['values']['5FF8'])
        self.accelY = np.array(data['values']['5FF9'])
        self.accelZ = np.array(data['values']['5FFA'])
        # Collapse the last two columns to one by 2-norm of the components
        self.accelX = np.sqrt(np.square(self.accelX[:,:,0]) + np.square(self.accelX[:, :, 1]))
        self.accelY = np.sqrt(np.square(self.accelY[:,:,0]) + np.square(self.accelY[:, :, 1]))
        self.accelZ = np.sqrt(np.square(self.accelZ[:,:,0]) + np.square(self.accelZ[:, :, 1]))
        # Close the file
        f.close()
        self.minCols = (min(self.accelX.shape[1], self.accelY.shape[1], self.accelZ.shape[1]))
        self.accelX  = np.transpose(self.accelX[:, 0: self.minCols])
        self.accelY  = np.transpose(self.accelY[:, 0: self.minCols])
        self.accelZ  = np.transpose(self.accelZ[:, 0: self.minCols])
        self.yData   = np.ones([self.accelY.shape[0], 1])*label
        self.featVecA = self.accelX + self.accelY + self.accelZ 
        self.featVecB  = np.dstack((self.accelX, self.accelY, self.accelZ))


In [4]:
# create empty objects
SMSDataObjNormal_1   = []
SMSDataObjNormal_2   = []
SMSDataObjabNormal_1 = []
SMSDataObjabNormal_2 = []

# define a class object to hold the data
SMSDataObjNormal_1 = SMSData("/content/sample_data/normal_1_6dB_Fan.json", 0)
SMSDataObjNormal_2 = SMSData("/content/sample_data/normal_2_6dB_Fan.json", 0)
SMSDataObjabNormal_1 = SMSData("/content/sample_data/abnormal_1_6dB_Fan.json", 1)
SMSDataObjabNormal_2 = SMSData("/content/sample_data/abnormal_2_6dB_Fan.json", 1)
SMSDataObjabNormal_4 = SMSData("/content/sample_data/abnormal_4HP_6dB_Fan.json", 1)
SMSDataObjNormal_5 = SMSData("/content/sample_data/normal_5_6dB_Fan.json", 0)


In [5]:
# for debug purpose
print('No of columns used', SMSDataObjabNormal_1.minCols)
print('Size of each axis', np.shape(SMSDataObjabNormal_1.accelX))
print('Feature vector B looks like', np.shape(SMSDataObjabNormal_1.featVecB))
minCols = min(SMSDataObjNormal_1.minCols, SMSDataObjNormal_2.minCols, SMSDataObjabNormal_1.minCols, SMSDataObjabNormal_1.minCols, SMSDataObjabNormal_4.minCols)
print('Minimum cols across the data is', minCols)

No of columns used 513
Size of each axis (513, 256)
Feature vector B looks like (513, 256, 3)
Minimum cols across the data is 512


In [6]:
# get data in x (input) and y (output)
x = np.concatenate((SMSDataObjNormal_1.featVecB, SMSDataObjabNormal_1.featVecB, 
                    SMSDataObjNormal_2.featVecB, SMSDataObjabNormal_2.featVecB,
                    SMSDataObjabNormal_4.featVecB, SMSDataObjNormal_5.featVecB))

y = np.concatenate((SMSDataObjNormal_1.yData, SMSDataObjabNormal_1.yData, 
                    SMSDataObjNormal_2.yData, SMSDataObjabNormal_2.yData,
                    SMSDataObjabNormal_4.yData, SMSDataObjNormal_5.yData))

# now shuffle the data, it doesn't matter anymore if these appear in order 
shuffle_in_unison_scary(x, y)

print(np.shape(x), np.shape(y))

(3078, 256, 3) (3078, 1)


In [7]:
trainIdx = 2051
validIdx = 2051
testIdx = 2651
endIdx = 3075
# generate a dataset for training and testing
x_train = x[1:trainIdx, :, :]
y_train = y[1:trainIdx, :]

x_valid = x[validIdx:testIdx, :, :]
y_valid = y[validIdx:testIdx, :]


x_test = x[testIdx:endIdx, :, :]
y_test = y[testIdx:endIdx, :]

# check vector shape
print(x_train.shape)
print(y_train.shape)

print(x_valid.shape)
print(y_valid.shape)

print(x_test.shape)
print(y_test.shape)

(2050, 256, 3)
(2050, 1)
(600, 256, 3)
(600, 1)
(424, 256, 3)
(424, 1)


In [135]:
# check vector shape
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)


(2200, 256, 3)
(2200, 1)
(359, 256, 3)
(359, 1)


In [29]:
# time to try NN

# model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(256, 3)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2)
])

# loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# settings
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
#model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [31]:
# train the dog
trgHist = model.fit(x_train, y_train, epochs=15, batch_size=16, validation_data=(x_valid, y_valid))

Epoch 1/15
129/129 [==============================] - 0s 2ms/step - loss: 0.0152 - accuracy: 0.9946 - val_loss: 0.1968 - val_accuracy: 0.9650
Epoch 2/15
129/129 [==============================] - 0s 2ms/step - loss: 0.0181 - accuracy: 0.9922 - val_loss: 0.2667 - val_accuracy: 0.9567
Epoch 3/15
129/129 [==============================] - 0s 2ms/step - loss: 0.0251 - accuracy: 0.9878 - val_loss: 0.1751 - val_accuracy: 0.9617
Epoch 4/15
129/129 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 0.9951 - val_loss: 0.2157 - val_accuracy: 0.9633
Epoch 5/15
129/129 [==============================] - 0s 2ms/step - loss: 0.0140 - accuracy: 0.9956 - val_loss: 0.1995 - val_accuracy: 0.9600
Epoch 6/15
129/129 [==============================] - 0s 2ms/step - loss: 0.0072 - accuracy: 0.9966 - val_loss: 0.2206 - val_accuracy: 0.9633
Epoch 7/15
129/129 [==============================] - 0s 2ms/step - loss: 0.0089 - accuracy: 0.9966 - val_loss: 0.2123 - val_accuracy: 0.9617
Epoch 

In [ ]:
# plot


In [32]:
# ask him to perform tricks
model.evaluate(x_test[1:400],  y_test[1:400], verbose=2)

13/13 - 0s - loss: 0.4691 - accuracy: 0.9373


[0.46906447410583496, 0.9373433589935303]

In [153]:
# what's the probability?
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[1:3])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.0000000e+00, 9.5147978e-10],
       [1.0178229e-07, 9.9999988e-01]], dtype=float32)>

In [116]:
predictNow = model(x_test[120:121]).numpy()
predictNow

array([[-2.6798193, -4.1256447]], dtype=float32)

In [33]:
SMSDataObjNormal_3 = SMSData("/content/sample_data/normal_3HP_6dB_Fan.json", 0)
print('Feature vector B looks like', np.shape(SMSDataObjNormal_3.featVecB))
x_test_a = SMSDataObjNormal_3.featVecB[1:trainIdx, :, :]
y_test_a = SMSDataObjNormal_3.yData[1:trainIdx, :]
print(np.shape(x_test_a), np.shape(y_test_a))                          

Feature vector B looks like (512, 256, 3)
(511, 256, 3) (511, 1)


In [34]:
model.evaluate(x_test_a,  y_test_a, verbose=2)

16/16 - 0s - loss: 3.0422 - accuracy: 0.8063


[3.04223370552063, 0.8062622547149658]

In [35]:
SMSDataObjabNormal_3 = SMSData("/content/sample_data/abnormal_3HP_6dB_Fan.json", 1)
print('Feature vector B looks like', np.shape(SMSDataObjabNormal_3.featVecB))
x_test_ab = SMSDataObjabNormal_3.featVecB[1:trainIdx, :, :]
y_test_ab = SMSDataObjabNormal_3.yData[1:trainIdx, :]
print(np.shape(x_test_ab), np.shape(y_test_ab))   

Feature vector B looks like (513, 256, 3)
(512, 256, 3) (512, 1)


In [36]:
model.evaluate(x_test_ab,  y_test_ab, verbose=2)

16/16 - 0s - loss: 4.6873 - accuracy: 0.8281


[4.687267780303955, 0.828125]

In [37]:
SMSDataObjNormal_4 = SMSData("/content/sample_data/normal_4HP_6dB_Fan.json", 0)
print('Feature vector B looks like', np.shape(SMSDataObjNormal_4.featVecB))
x_test_ab = SMSDataObjNormal_4.featVecB[1:trainIdx, :, :]
y_test_ab = SMSDataObjNormal_4.yData[1:trainIdx, :]
print(np.shape(x_test_ab), np.shape(y_test_ab))  

Feature vector B looks like (513, 256, 3)
(512, 256, 3) (512, 1)


In [38]:
model.evaluate(x_test_ab,  y_test_ab, verbose=2)

16/16 - 0s - loss: 0.9417 - accuracy: 0.9395


[0.941677451133728, 0.939453125]